In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from shapash.explainer.smart_explainer import SmartExplainer
import pandas as pd
import joblib
import os

/Users/lilya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [22]:
train_path = os.path.join('..', 'src', 'models', 'series_train.csv')
test_path = os.path.join('..', 'src', 'models', 'series_test.csv')
series_train = pd.read_csv(train_path)
series_test = pd.read_csv(test_path)

#--Prophet--
series_train = series_train.rename(columns={'date': 'ds', 'sales': 'y'})
series_test = series_test.rename(columns={'date': 'ds', 'sales': 'y'})
#--au cas ou--
series_train['ds'] = pd.to_datetime(series_train['ds']) 
series_test['ds'] = pd.to_datetime(series_test['ds'])
series_train['y'] = series_train['y'].astype(float)
series_test['y'] = series_test['y'].astype(float)

X_train = series_train.drop(columns=['y'])
y_train = series_train['y']
X_test = series_test.drop(columns=['y'])
y_test = series_test['y']

feature_names = ['ds', 'onpromotion', 'oil_price', 'holiday']

In [23]:
model_path = os.path.join('..', 'src', 'models', 'prophet_model_exogs.pkl')
model = joblib.load(model_path)

In [24]:
X_test_df = pd.DataFrame(X_test, columns=feature_names)

X_test_df.head()

,ds,onpromotion,oil_price,holiday
0,2016-01-01,159,37.076647,1
1,2016-01-02,1832,36.970000,0
2,2016-01-03,2936,36.863353,0
3,2016-01-04,1741,36.810000,0
4,2016-01-05,3950,35.970000,0


In [26]:
explainer = SmartExplainer(model=model)

explainer.compile(x=X_test_df.drop(columns=['ds']), y_target=pd.Series(y_test, index=X_test_df.index, name="y"))

app = explainer.run_app(title_story='time series', port=8020)

KeyError: 'ds'

In [31]:
app.kill()

In [13]:
if 'ds' in X_test_df.columns:
    X_test_df['ds'] = pd.to_datetime(X_test_df['ds']).astype(int) / 10**9  # Convertir en timestamp (secondes)

In [15]:
X_test_df = pd.get_dummies(X_test_df, drop_first=True)  # Codage one-hot

In [24]:
X_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ds           608 non-null    datetime64[ns]
 1   onpromotion  608 non-null    int64         
 2   oil_price    608 non-null    float64       
 3   holiday      608 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 19.1 KB


In [20]:
# Conversion explicite des colonnes pour garantir la compatibilité
X_test_df = X_test_df.astype({
    'ds': 'float64',
    'onpromotion': 'int64',
    'oil_price': 'float64',
    'holiday': 'int64'
})

In [33]:

test_path = os.path.join('..', 'src', 'models', 'series_test.csv')

series_test = pd.read_csv(test_path)

#--Prophet--

series_test = series_test.rename(columns={'date': 'ds', 'sales': 'y'})
#--au cas ou--

series_test['ds'] = pd.to_datetime(series_test['ds'])

series_test['y'] = series_test['y'].astype(float)

X_test = series_test.drop(columns=['y'])
y_test = series_test['y']


In [34]:
import os
import joblib
import pandas as pd

# Charger le modèle
model_path = os.path.join('..', 'src', 'models', 'prophet_model_exogs.pkl')
model = joblib.load(model_path)

# Charger les données associées (assure-toi d'avoir les mêmes exogènes qu'à l'entraînement)
# Exemple de chemin vers les données
data = series_test

# Vérifier les colonnes nécessaires
print(data.head())

          ds             y  onpromotion  oil_price  holiday  \
0 2016-01-01  1.643339e+04          159  37.076647        1   
1 2016-01-02  1.066677e+06         1832  36.970000        0   
2 2016-01-03  1.226736e+06         2936  36.863353        0   
3 2016-01-04  9.559569e+05         1741  36.810000        0   
4 2016-01-05  8.353204e+05         3950  35.970000        0   

   Test_Forecast_Sarimax  Test_Forecast_prophet  test_Forecast_LSTM  \
0           8.832907e+05           8.467677e+05                 NaN   
1           6.229928e+05           1.013931e+06                 NaN   
2           1.115351e+06           1.064571e+06                 NaN   
3           9.056684e+05           8.313662e+05                 NaN   
4           9.304216e+05           8.027423e+05                 NaN   

   test_Forecast_XGB  test_Forecast_RF  test_Forecast_prophet_online  
0          334607.97     386969.305608                 690101.175402  
1          334607.97     825807.859319              

In [36]:
# Définir les colonnes exogènes et la date
exog_columns = ['onpromotion', 'oil_price', 'holiday']  # Tes exogènes
date_column = 'date'

# Préparer les données d'entrée
future = data[['ds'] + exog_columns]
future['ds'] = pd.to_datetime(future['ds'])

# Afficher les données d'exogènes
print(future.head())

          ds  onpromotion  oil_price  holiday
0 2016-01-01          159  37.076647        1
1 2016-01-02         1832  36.970000        0
2 2016-01-03         2936  36.863353        0
3 2016-01-04         1741  36.810000        0
4 2016-01-05         3950  35.970000        0


/var/folders/db/swcwxrj54c30_jn72llv15kh0000gn/T/ipykernel_1894/3880885861.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
# Faire des prédictions
forecast = model.predict(future)

# Vérifier les prédictions
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

          ds          yhat     yhat_lower    yhat_upper
0 2016-01-01  8.476734e+05  694122.579440  9.942190e+05
1 2016-01-02  1.016574e+06  866129.870881  1.168742e+06
2 2016-01-03  1.066906e+06  911922.565215  1.225720e+06
3 2016-01-04  8.324834e+05  685582.708261  9.893693e+05
4 2016-01-05  8.040278e+05  644888.740774  9.619267e+05


In [38]:
from shapash.explainer.smart_explainer import SmartExplainer

# Initialiser SHAPASH avec les données exogènes
xpl = SmartExplainer(model=model, features_dict=dict(zip(future[exog_columns].columns, exog_columns)))
xpl.compile(x=future[exog_columns], y_pred=forecast['yhat'])

# Afficher le dashboard interactif
xpl.show()

INFO: Shap explainer type - <shap.explainers._exact.ExactExplainer object at 0x314089f10>


KeyError: 'ds'

In [ ]:
xpl.compile()

In [39]:
print(future.head())
print(future.columns)

          ds  onpromotion  oil_price  holiday
0 2016-01-01          159  37.076647        1
1 2016-01-02         1832  36.970000        0
2 2016-01-03         2936  36.863353        0
3 2016-01-04         1741  36.810000        0
4 2016-01-05         3950  35.970000        0
Index(['ds', 'onpromotion', 'oil_price', 'holiday'], dtype='object')


In [41]:

print(forecast.columns)

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'annual', 'annual_lower', 'annual_upper', 'extra_regressors_additive',
       'extra_regressors_additive_lower', 'extra_regressors_additive_upper',
       'holiday', 'holiday_lower', 'holiday_upper', 'oil_price',
       'oil_price_lower', 'oil_price_upper', 'onpromotion',
       'onpromotion_lower', 'onpromotion_upper', 'weekly', 'weekly_lower',
       'weekly_upper', 'yearly', 'yearly_lower', 'yearly_upper',
       'multiplicative_terms', 'multiplicative_terms_lower',
       'multiplicative_terms_upper', 'yhat'],
      dtype='object')


In [42]:
import shap

explainer = shap.Explainer(model.predict, future.drop(columns=['ds']))  # Exclure 'ds'
shap_values = explainer(future.drop(columns=['ds']))
shap_df = pd.DataFrame(shap_values.values, columns=future.drop(columns=['ds']).columns, index=future.index)

KeyError: 'ds'

In [45]:
from shapash.explainer.smart_explainer import SmartExplainer

xpl = SmartExplainer(model=model, features_dict=dict(zip(future.drop(columns=['ds']).columns, future.drop(columns=['ds']).columns)))
shap_values = explainer(future.drop(columns=['ds']))
shap_df = pd.DataFrame(shap_values.values, columns=future.drop(columns=['ds']).columns, index=future.index)
# Compiler Shapash avec les données correctes
xpl.compile(
    x=future.drop(columns=['ds']),  # Exclure la colonne 'ds'
    y_pred=forecast['yhat'],       # Colonne des prédictions
    contributions=shap_df          # Contributions (SHAP values)
)

KeyError: 'ds'